In [1]:
import numpy as np
from scipy.signal import correlate2d
import tensorflow.keras as keras
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score

# Класс для сверточного слоя
class Convolution:

    def __init__(self, inputShape, filterSize, numFilters):
        inputHeight, inputWidth = inputShape
        self.numFilters = numFilters
        self.inputShape = inputShape

        # Размеры выходных данных и фильтров
        self.filterShape = (numFilters, filterSize, filterSize)
        self.outputShape = (numFilters, inputHeight - filterSize + 1, inputWidth - filterSize + 1)

        self.filters = np.random.randn(*self.filterShape)
        self.biases = np.random.randn(*self.outputShape)

    # Прямой проход
    def forward(self, inputData):
        self.inputData = inputData
        output = np.zeros(self.outputShape)
        for i in range(self.numFilters):
            output[i] = correlate2d(self.inputData, self.filters[i], mode="valid")
        output = np.maximum(output, 0)
        return output

    # Обратный проход
    def backward(self, dL_dout, lr):
        dL_dinput = np.zeros_like(self.inputData)
        dL_dfilters = np.zeros_like(self.filters)

        for i in range(self.numFilters):
            dL_dfilters[i] = correlate2d(self.inputData, dL_dout[i],mode="valid")
            dL_dinput += correlate2d(dL_dout[i],self.filters[i], mode="full")

        self.filters -= lr * dL_dfilters
        self.biases -= lr * dL_dout

        return dL_dinput

# Класс для слоя максимального пулинга
class MaxPool:

    def __init__(self, poolSize):
        self.poolSize = poolSize

    # Прямой проход
    def forward(self, inputData):
        self.inputData = inputData
        self.numChannels, self.inputHeight, self.inputWidth = inputData.shape
        self.outputHeight = self.inputHeight // self.poolSize
        self.outputWidth = self.inputWidth // self.poolSize

        self.output = np.zeros((self.numChannels, self.outputHeight, self.outputWidth))

        for c in range(self.numChannels):
            for i in range(self.outputHeight):
                for j in range(self.outputWidth):
                    startI = i * self.poolSize
                    startJ = j * self.poolSize
                    endI = startI + self.poolSize
                    endJ = startJ + self.poolSize
                    patch = inputData[c, startI:endI, startJ:endJ]
                    self.output[c, i, j] = np.max(patch)

        return self.output

    # Обратный проход
    def backward(self, dL_dout, lr):
        dL_dinput = np.zeros_like(self.inputData)

        for c in range(self.numChannels):
            for i in range(self.outputHeight):
                for j in range(self.outputWidth):
                    startI = i * self.poolSize
                    startJ = j * self.poolSize
                    endI = startI + self.poolSize
                    endJ = startJ + self.poolSize
                    patch = self.inputData[c, startI:endI, startJ:endJ]
                    mask = patch == np.max(patch)
                    dL_dinput[c,startI:endI, startJ:endJ] = dL_dout[c, i, j] * mask

        return dL_dinput

# Класс для полносвязного слоя
class FullyConnected:

    def __init__(self, inputSize, outputSize):
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.weights = np.random.randn(outputSize, self.inputSize)
        self.biases = np.random.rand(outputSize, 1)

    # Функция активации softmax
    def softmax(self, z):
        shiftedZ = z - np.max(z)
        expValues = np.exp(shiftedZ)
        sumExpValues = np.sum(expValues, axis=0)
        probabilities = expValues / sumExpValues
        return probabilities

    # Производная softmax
    def softmaxDerivative(self, s):
        return np.diagflat(s) - np.dot(s, s.T)

    # Прямой проход
    def forward(self, inputData):
        self.inputData = inputData
        flattenedInput = inputData.flatten().reshape(1, -1)
        self.z = np.dot(self.weights, flattenedInput.T) + self.biases
        self.output = self.softmax(self.z)
        return self.output

    # Обратный проход
    def backward(self, dL_dout, lr):
        dL_dy = np.dot(self.softmaxDerivative(self.output), dL_dout)
        dL_dw = np.dot(dL_dy, self.inputData.flatten().reshape(1, -1))
        dL_db = dL_dy
        dL_dinput = np.dot(self.weights.T, dL_dy)
        dL_dinput = dL_dinput.reshape(self.inputData.shape)
        self.weights -= lr * dL_dw
        self.biases -= lr * dL_db
        return dL_dinput

# Функция потерь - кросс-энтропия
def crossEntropyLoss(predictions, targets):
    numSamples = 10
    epsilon = 1e-7
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    loss = -np.sum(targets * np.log(predictions)) / numSamples
    return loss

# Градиент функции потерь
def crossEntropyLossGradient(actualLabels, predictedProbs):
    numSamples = actualLabels.shape[0]
    gradient = -actualLabels / (predictedProbs + 1e-7) / numSamples
    return gradient

# Обучение сети
def trainNetwork(X, y, conv, pool, full, lr=0.01, epochs=30):
    for epoch in range(epochs):
        totalLoss = 0.0
        correctPredictions = 0

        for i in range(len(X)):
            convOut = conv.forward(X[i])
            poolOut = pool.forward(convOut)
            fullOut = full.forward(poolOut)
            loss = crossEntropyLoss(fullOut.flatten(), y[i])
            totalLoss += loss

            oneHotPred = np.zeros_like(fullOut)
            oneHotPred[np.argmax(fullOut)] = 1
            oneHotPred = oneHotPred.flatten()

            numPred = np.argmax(oneHotPred)
            numY = np.argmax(y[i])

            if numPred == numY:
                correctPredictions += 1

            gradient = crossEntropyLossGradient(y[i], fullOut.flatten()).reshape((-1, 1))
            fullBack = full.backward(gradient, lr)
            poolBack = pool.backward(fullBack, lr)
            convBack = conv.backward(poolBack, lr)

        averageLoss = totalLoss / len(X)
        accuracy = correctPredictions / len(X_train) * 100.0
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {averageLoss:.4f} - Accuracy: {accuracy:.2f}%")

# Предсказание
def predict(inputSample, conv, pool, full):
    convOut = conv.forward(inputSample)
    poolOut = pool.forward(convOut)
    flattenedOutput = poolOut.flatten()
    predictions = full.forward(flattenedOutput)
    return predictions

# Загрузка данных
(trainImages, trainLabels), (testImages, testLabels) = keras.datasets.fashion_mnist.load_data()
X_train = trainImages[:5000] / 255.0
y_train = trainLabels[:5000]

X_test = trainImages[5000:10000] / 255.0
y_test = trainLabels[5000:10000]

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

conv = Convolution(X_train[0].shape, 6, 1)
pool = MaxPool(2)
full = FullyConnected(121, 10)

trainNetwork(X_train, y_train, conv, pool, full)

predictions = []
for data in X_test:
    pred = predict(data, conv, pool, full)
    oneHotPred = np.zeros_like(pred)
    oneHotPred[np.argmax(pred)] = 1
    predictions.append(oneHotPred.flatten())

predictions = np.array(predictions)

accuracy_score(predictions, y_test)

2023-12-22 22:42:07.577213: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-22 22:42:07.578738: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-22 22:42:07.602348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-22 22:42:07.602376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-22 22:42:07.603305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

4422102/4422102 [==============================] - 1s 0us/step
Epoch 1/30 - Loss: 1.1736 - Accuracy: 23.44%
Epoch 2/30 - Loss: 1.0641 - Accuracy: 30.42%
Epoch 3/30 - Loss: 0.9901 - Accuracy: 31.64%
Epoch 4/30 - Loss: 0.8911 - Accuracy: 35.70%
Epoch 5/30 - Loss: 0.8354 - Accuracy: 29.62%
Epoch 6/30 - Loss: 0.9286 - Accuracy: 39.54%
Epoch 7/30 - Loss: 0.9030 - Accuracy: 41.70%
Epoch 8/30 - Loss: 0.8922 - Accuracy: 42.60%
Epoch 9/30 - Loss: 0.8823 - Accuracy: 43.02%
Epoch 10/30 - Loss: 0.8760 - Accuracy: 43.72%
Epoch 11/30 - Loss: 0.8693 - Accuracy: 44.06%
Epoch 12/30 - Loss: 0.8642 - Accuracy: 44.52%
Epoch 13/30 - Loss: 0.8607 - Accuracy: 44.68%
Epoch 14/30 - Loss: 0.8585 - Accuracy: 44.88%
Epoch 15/30 - Loss: 0.8567 - Accuracy: 44.78%
Epoch 16/30 - Loss: 0.8542 - Accuracy: 45.00%
Epoch 17/30 - Loss: 0.8520 - Accuracy: 45.24%
Epoch 18/30 - Loss: 0.8498 - Accuracy: 45.56%
Epoch 19/30 - Loss: 0.8485 - Accuracy: 45.68%
Epoch 20/30 - Loss: 0.8474 - Accuracy: 45.78%
Epoch 21/30 - Loss: 0.8464

0.4506